# Rf 1 0 8_Plotbinning
Basic functionality: plotting unbinned data with alternate and variable binnings




**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:41 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussModel.h"
#include "RooDecay.h"
#include "RooBMixDecay.h"
#include "RooCategory.h"
#include "RooBinning.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup model
 ---------------------

Build a b decay p.d.f with mixing

In [3]:
RooRealVar dt("dt", "dt", -20, 20);
RooRealVar dm("dm", "dm", 0.472);
RooRealVar tau("tau", "tau", 1.547);
RooRealVar w("w", "mistag rate", 0.1);
RooRealVar dw("dw", "delta mistag rate", 0.);

RooCategory mixState("mixState", "B0/B0bar mixing state");
mixState.defineType("mixed", -1);
mixState.defineType("unmixed", 1);
RooCategory tagFlav("tagFlav", "Flavour of the tagged B0");
tagFlav.defineType("B0", 1);
tagFlav.defineType("B0bar", -1);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Build a gaussian resolution model

In [4]:
RooRealVar bias1("bias1", "bias1", 0);
RooRealVar sigma1("sigma1", "sigma1", 0.1);
RooGaussModel gm1("gm1", "gauss model 1", dt, bias1, sigma1);

Construct bdecay (x) gauss

In [5]:
RooBMixDecay bmix("bmix", "decay", dt, mixState, tagFlav, tau, dm, w, dw, gm1, RooBMixDecay::DoubleSided);

Sample data from model
 --------------------------------------------

Sample 2000 events in (dt,mixstate,tagflav) from bmix

In [6]:
RooDataSet *data = bmix.generate(RooArgSet(dt, mixState, tagFlav), 2000);

Show dt distribution with custom binning
 -------------------------------------------------------------------------------

Make plot of dt distribution of data in range (-15,15) with fine binning for dt>0 and coarse binning for dt<0

Create binning object with range (-15,15)

In [7]:
RooBinning tbins(-15, 15);

Add 60 bins with uniform spacing in range (-15,0)

In [8]:
tbins.addUniform(60, -15, 0);

Add 15 bins with uniform spacing in range (0,15)

In [9]:
tbins.addUniform(15, 0, 15);

Make plot with specified binning

In [10]:
RooPlot *dtframe = dt.frame(Range(-15, 15), Title("dt distribution with custom binning"));
data->plotOn(dtframe, Binning(tbins));
bmix.plotOn(dtframe);

[#1] INFO:Plotting -- RooAbsReal::plotOn(bmix) plot on dt integrates over variables (mixState,tagFlav)


Nb: note that bin density for each bin is adjusted to that of default frame binning as shown
 in Y axis label (100 bins --> Events/0.4*Xaxis-dim) so that all bins represent a consistent density distribution

Show mixstate asymmetry with custom binning
 ------------------------------------------------------------------------------------

Make plot of dt distribution of data asymmetry in 'mixstate' with variable binning

Create binning object with range (-10,10)

In [11]:
RooBinning abins(-10, 10);

Add boundaries at 0, (-1,1), (-2,2), (-3,3), (-4,4) and (-6,6)

In [12]:
abins.addBoundary(0);
abins.addBoundaryPair(1);
abins.addBoundaryPair(2);
abins.addBoundaryPair(3);
abins.addBoundaryPair(4);
abins.addBoundaryPair(6);

Create plot frame in dt

In [13]:
RooPlot *aframe = dt.frame(Range(-10, 10), Title("mixState asymmetry distribution with custom binning"));

Plot mixstate asymmetry of data with specified custom binning

In [14]:
data->plotOn(aframe, Asymmetry(mixState), Binning(abins));

Plot corresponding property of p.d.f

In [15]:
bmix.plotOn(aframe, Asymmetry(mixState));

[#1] INFO:Plotting -- RooAbsReal::plotAsymOn(bmix) plot on dt projects variables (tagFlav)


Adjust vertical range of plot to sensible values for an asymmetry

In [16]:
aframe->SetMinimum(-1.1);
aframe->SetMaximum(1.1);

Nb: for asymmetry distributions no density corrects are needed (and are thus not applied)

Draw plots on canvas

In [17]:
TCanvas *c = new TCanvas("rf108_plotbinning", "rf108_plotbinning", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
dtframe->GetYaxis()->SetTitleOffset(1.6);
dtframe->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
aframe->GetYaxis()->SetTitleOffset(1.6);
aframe->Draw();

Draw all canvases 

In [18]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()